<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/notebooks/temp/analyse30repoPOO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!rm -rf repositorio*

# Classes

## Dependências

In [1]:
!pip3 install gitpython


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import requests
import pandas as pd
import os
import threading
import datetime
from git import Repo

class GitHubRepoChecker:
    def __init__(self, url):
        self.url = url
        self.df_repositorios = None
        self.lista_urls_validas = []
        self.lista_urls_invalidas = []

    def check_url(self, url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return True
            else:
                return False
        except requests.exceptions.RequestException:
            return False

    def load_data(self):
        self.df_repositorios = pd.read_csv(filepath_or_buffer=self.url, sep=';')

    def validate_urls(self):
        if self.df_repositorios is None:
            raise ValueError("Os dados não foram carregados. Chame o método 'load_data()' primeiro.")

        lista_url_repositorios = self.df_repositorios['Git Repository'].tolist()

        for url in lista_url_repositorios:
            url_valida = self.check_url(url)
            if url_valida:
                self.lista_urls_validas.append(url)
            else:
                self.lista_urls_invalidas.append(url)

    def get_invalid_urls(self):
        return self.lista_urls_invalidas

    def get_valid_urls(self):
        return self.lista_urls_validas


class RepoCloner:
    def __init__(self, qtd_repositorios, lista_urls):
        self.qtd_repositorios = qtd_repositorios
        self.lista_urls = lista_urls
        self.lista_threads = []
        self.lista_tempo_inicio = []
        self.lista_tempo_conclusao = []

    def my_clone(self, git_url, repo_dir):
        try:
            Repo.clone_from(git_url, repo_dir)
            tempo = datetime.datetime.now()
            print(f'Clonagem do repo {git_url} concluída com sucesso! -> instante: {tempo}')
            elemento = (repo_dir, tempo)
            self.lista_tempo_conclusao.append(elemento)
        except Exception as ex:
            print(f"Erro ao clonar o repo: {git_url} - {str(ex)}")

    def clona_repositorios(self):
        for i in range(self.qtd_repositorios):
            nome = 'repositorio'+str(i+1)
            tdi = threading.Thread(target=self.my_clone, args=(self.lista_urls[i], nome))
            tdi.name = 'Thread-'+str(i+1)
            inicio_thread = datetime.datetime.now()
            print(f"Thread {i+1} iniciada em {inicio_thread} - clonando {nome}:{self.lista_urls[i]}...")
            self.lista_threads.append(tdi)
            elemento = (nome, inicio_thread)
            self.lista_tempo_inicio.append(elemento)
            tdi.start()

        for each in self.lista_threads:
            each.join()

    def mostra_tempo_clonagens(self):
        for t1 in self.lista_tempo_inicio:
            for t2 in self.lista_tempo_conclusao:
                if t1[0] == t2[0]:
                    print(f"Tempo de download do {t1[0]} é: {t2[1]-t1[1]} s")


class RepositorioChecker:
    def __init__(self, url_repositorios_src, url_complemento):
        self.df_repositorios_src = pd.read_csv(url_repositorios_src)
        self.df_complemento = pd.read_csv(url_complemento)
        self.df_repositorios = None

    def merge_dataframes(self):
        self.df_complemento['nome'] = self.df_repositorios_src['projeto'].to_numpy()
        self.df_complemento['source_java'] = self.df_repositorios_src['source'].to_numpy()
        self.df_repositorios = self.df_complemento.copy()
        self.df_repositorios['source_java'] = self.df_repositorios['source_java'].str.replace('.', '/')
        self.df_repositorios = self.df_repositorios[['nome', 'repositorio', 'source_java', 'url']]
        self.df_repositorios.to_csv('l4prepositoriossrc.csv', index=False)

    def check_sub_path(self, sub_path, repositorio):
        current_path = os.getcwd()
        abs_sub_path = os.path.join(current_path, repositorio, sub_path)
        print(f"Analisa {sub_path} em {repositorio} - {os.path.exists(abs_sub_path)}")
        return os.path.exists(abs_sub_path)

    def check_repositorios(self):
        if self.df_repositorios is None:
            raise ValueError("Os dados não foram carregados. Chame o método 'merge_dataframes()' primeiro.")

        lista_nomes_repositorio = self.df_repositorios['nome'].tolist()
        lista_repositorios = self.df_repositorios['repositorio'].tolist()
        lista_src = self.df_repositorios['source_java'].tolist()
        lista_checa = []
        lista_urls = self.df_repositorios['url'].to_list()

        for i, repositorio in enumerate(lista_repositorios):
            resultado = self.check_sub_path(sub_path=lista_src[i], repositorio=repositorio)
            lista_checa.append(resultado)

        dict_temp = {
            'nome': lista_nomes_repositorio,
            'repositorio': lista_repositorios,
            'url': lista_urls,
            'src': lista_src,
            'checa': lista_checa
        }

        df_checa_repositorios = pd.DataFrame(dict_temp)
        return df_checa_repositorios

# 1. Carrega os dados dos repositórios

In [4]:
# URLs dos repositórios
URL = 'https://raw.githubusercontent.com/armandossrecife/teste/main/listaderepositoriosprincipais.csv'

# Verifica a validade das URLs dos repositórios
checker = GitHubRepoChecker(URL)
checker.load_data()
checker.validate_urls()

invalid_urls = checker.get_invalid_urls()
valid_urls = checker.get_valid_urls()

print(f'URLs inválidas:')
for url in invalid_urls:
    print(url)

print(f'URLs válidas:')
for url in valid_urls:
    print(url)

URLs inválidas:
URLs válidas:
https://github.com/apache/cassandra.git
https://github.com/apache/kafka.git
https://github.com/spring-projects/spring-boot.git
https://github.com/elastic/elasticsearch.git


## Dados dos 4 repositórios principais

In [5]:
df_repositorios = checker.df_repositorios
df_repositorios

,Category,Project,Git Repository,Description
0,Data storage and management,cassandra,https://github.com/apache/cassandra.git,Distributed NoSQL database
1,Message broker,kafka,https://github.com/apache/kafka.git,Message broker
2,Web engines and web tools,spring-boot,https://github.com/spring-projects/spring-boot...,Spring-based project manager
3,Search engines,elasticsearch,https://github.com/elastic/elasticsearch.git,Search engine


## 2. Clona os respositórios

In [6]:
# Clona os repositórios
lista_url_repositorios = checker.get_valid_urls()
qtd_repositorios = len(lista_url_repositorios)

print(f'Clonando os {qtd_repositorios} repositórios. Aguarde...')
cloner = RepoCloner(qtd_repositorios, lista_url_repositorios)
cloner.clona_repositorios()
cloner.mostra_tempo_clonagens()

Clonando os 4 repositórios. Aguarde...
Thread 1 iniciada em 2023-08-17 17:01:36.401374 - clonando repositorio1:https://github.com/apache/cassandra.git...
Thread 2 iniciada em 2023-08-17 17:01:36.403032 - clonando repositorio2:https://github.com/apache/kafka.git...
Thread 3 iniciada em 2023-08-17 17:01:36.405485 - clonando repositorio3:https://github.com/spring-projects/spring-boot.git...
Thread 4 iniciada em 2023-08-17 17:01:36.407396 - clonando repositorio4:https://github.com/elastic/elasticsearch.git...
Clonagem do repo https://github.com/apache/kafka.git concluída com sucesso! -> instante: 2023-08-17 17:05:14.432291
Clonagem do repo https://github.com/spring-projects/spring-boot.git concluída com sucesso! -> instante: 2023-08-17 17:06:58.665210
Clonagem do repo https://github.com/apache/cassandra.git concluída com sucesso! -> instante: 2023-08-17 17:07:17.340486
Clonagem do repo https://github.com/elastic/elasticsearch.git concluída com sucesso! -> instante: 2023-08-17 17:09:26.5677

# 3. Recupera informação do diretório src/main

In [10]:
URL_DATASET_SRC_REPOSITORIOS = 'https://raw.githubusercontent.com/armandossrecife/teste/main/l4prepositorios.csv'
URL_DATASET_CLONAGEM_REPOSITORIOS = 'https://raw.githubusercontent.com/armandossrecife/teste/main/l4prepositoriosurl.csv'

# Verifica a existência de subdiretórios nos repositórios
repositorio_checker = RepositorioChecker(URL_DATASET_SRC_REPOSITORIOS, URL_DATASET_CLONAGEM_REPOSITORIOS)

repositorio_checker.merge_dataframes()
df_checa_repositorios = repositorio_checker.check_repositorios()

# Exibe o resultado da verificação
df_checa_repositorios

Analisa src/java em repositorio1 - True
Analisa core/src/main/java/kafka em repositorio2 - True
Analisa spring-boot-project/spring-boot/src/main/java/org/springframework/boot em repositorio3 - True
Analisa server/src/main/java/org/elasticsearch em repositorio4 - True


,nome,repositorio,url,src,checa
0,cassandra,repositorio1,https://github.com/apache/cassandra.git,src/java,True
1,kafka,repositorio2,https://github.com/apache/kafka.git,core/src/main/java/kafka,True
2,spring-boot,repositorio3,https://github.com/spring-projects/spring-boot...,spring-boot-project/spring-boot/src/main/java/...,True
3,elasticsearch,repositorio4,https://github.com/elastic/elasticsearch.git,server/src/main/java/org/elasticsearch,True


In [11]:
df_checa_repositorios.to_csv('repositorios_analise_tese.csv', index=False)